In [1]:
from __future__ import unicode_literals, print_function, division
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from preprocessing import get_dataframes
train_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/simple_split/tasks_train_simple.txt"
test_url = "https://raw.githubusercontent.com/brendenlake/SCAN/master/simple_split/tasks_test_simple.txt"
train_df, test_df = get_dataframes(train_url, test_url)

train_df.head()

,IN,OUT
0,"[jump, opposite, right, twice, and, turn, oppo...","[<sos>, I_TURN_RIGHT, I_TURN_RIGHT, I_JUMP, I_..."
1,"[run, opposite, left, after, walk, right, <eos>]","[<sos>, I_TURN_RIGHT, I_WALK, I_TURN_LEFT, I_T..."
2,"[walk, after, run, around, right, twice, <eos>]","[<sos>, I_TURN_RIGHT, I_RUN, I_TURN_RIGHT, I_R..."
3,"[look, around, right, thrice, and, turn, left,...","[<sos>, I_TURN_RIGHT, I_LOOK, I_TURN_RIGHT, I_..."
4,"[walk, opposite, left, twice, and, walk, oppos...","[<sos>, I_TURN_LEFT, I_TURN_LEFT, I_WALK, I_TU..."


In [3]:
from preprocessing import get_vocab

train_vocab_in = get_vocab(train_df['IN'], 'in')
train_vocab_out = get_vocab(train_df['OUT'], 'out')

print(train_vocab_in.word2index)
print(train_vocab_out.word2index)

def series2idx(row):
    x = train_vocab_in.col2idx(row, 'IN')
    y = train_vocab_out.col2idx(row, 'OUT')
    return x, y
    
train_df[['IN_idx','OUT_idx']] = train_df.apply(series2idx, axis=1, result_type='expand')

train_df.head()

{'<sos>': 0, '<eos>': 1, 'jump': 2, 'opposite': 3, 'right': 4, 'twice': 5, 'and': 6, 'turn': 7, 'thrice': 8, 'run': 9, 'left': 10, 'after': 11, 'walk': 12, 'around': 13, 'look': 14}
{'<sos>': 0, '<eos>': 1, 'I_TURN_RIGHT': 2, 'I_JUMP': 3, 'I_WALK': 4, 'I_TURN_LEFT': 5, 'I_RUN': 6, 'I_LOOK': 7}


,IN,OUT,IN_idx,OUT_idx
0,"[jump, opposite, right, twice, and, turn, oppo...","[<sos>, I_TURN_RIGHT, I_TURN_RIGHT, I_JUMP, I_...","[tensor(2), tensor(3), tensor(4), tensor(5), t...","[tensor(0), tensor(2), tensor(2), tensor(3), t..."
1,"[run, opposite, left, after, walk, right, <eos>]","[<sos>, I_TURN_RIGHT, I_WALK, I_TURN_LEFT, I_T...","[tensor(9), tensor(3), tensor(10), tensor(11),...","[tensor(0), tensor(2), tensor(4), tensor(5), t..."
2,"[walk, after, run, around, right, twice, <eos>]","[<sos>, I_TURN_RIGHT, I_RUN, I_TURN_RIGHT, I_R...","[tensor(12), tensor(11), tensor(9), tensor(13)...","[tensor(0), tensor(2), tensor(6), tensor(2), t..."
3,"[look, around, right, thrice, and, turn, left,...","[<sos>, I_TURN_RIGHT, I_LOOK, I_TURN_RIGHT, I_...","[tensor(14), tensor(13), tensor(4), tensor(8),...","[tensor(0), tensor(2), tensor(7), tensor(2), t..."
4,"[walk, opposite, left, twice, and, walk, oppos...","[<sos>, I_TURN_LEFT, I_TURN_LEFT, I_WALK, I_TU...","[tensor(12), tensor(3), tensor(10), tensor(5),...","[tensor(0), tensor(5), tensor(5), tensor(4), t..."


In [4]:
from dataloading import Text_dataset
from torch.utils.data import DataLoader

# Create datasets
training_data = Text_dataset(train_df[['IN_idx', 'OUT_idx']], True, 100000)
#test_data = Text_dataset(test_df, False)

# Define dataloaders
bs = 1 # Batch size
train_dataloader = DataLoader(training_data, batch_size=bs, shuffle=False)
#test_dataloader = DataLoader(test_data, batch_size=bs, shuffle=False)

In [5]:
for data in train_dataloader:
    xs, ys = data
    print(xs)
    print(ys)
    break

tensor([[ 7,  3, 10,  8,  6, 14, 13,  4,  5,  1]])
tensor([[0, 5, 5, 5, 5, 5, 5, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 2, 7, 1]])


In [7]:
from models import EncoderRNN, DecoderRNN
from training import train
max_len = 10
hidden_size = 200

encoder = EncoderRNN(train_vocab_in.n_words, hidden_size).to(device)
decoder = DecoderRNN(hidden_size, train_vocab_out.n_words, max_len, device).to(device)

train(train_dataloader, encoder, decoder)

AssertionError: Torch not compiled with CUDA enabled

tensor([[ 9, 10, 11,  7, 13, 10,  1]])
tensor([[[-0.2314,  0.6078,  0.2509, -1.4611,  1.8004, -0.6848, -1.1059,
          -0.5177, -0.8370,  0.0650],
         [-0.0566,  0.8017,  0.5498, -0.2374, -1.0558, -1.6312,  0.2324,
          -0.1150,  1.5447, -0.0980],
         [-0.4974,  0.4541,  0.4621, -0.2882,  1.4121, -1.2635,  0.1119,
           1.1863,  1.2696,  0.9653],
         [-1.2161, -0.7304, -0.8373, -0.2299, -0.6048,  0.6078, -0.9567,
           2.1896,  0.6506, -0.3528],
         [-1.1912,  0.5624, -1.0473,  0.3324,  0.8719, -1.4345,  0.0536,
           0.1425, -1.4736, -0.1098],
         [-0.0566,  0.8017,  0.5498, -0.2374, -1.0558, -1.6312,  0.2324,
          -0.1150,  1.5447, -0.0980],
         [ 0.8730, -0.2601, -0.6206,  0.3851, -1.2088,  0.6031,  1.5122,
           1.4980, -0.2072, -0.0791]]], grad_fn=<EmbeddingBackward0>)
torch.Size([1, 7, 10])
